In [1]:
import pandas as pd
from pandas_profiling import ProfileReport

In [9]:
financial = pd.read_csv("../dataset/재무데이터.txt", sep = '\t', encoding='cp949')
non_financial = pd.read_csv('../dataset/merge.csv')

In [10]:
len(financial['사업자번호'].unique())
financial.columns


Index(['사업자번호', '결산년월', '유동자산', '매출채권', '비유동자산', '유형자산', '자산총계', '유동부채',
       '비유동부채', '부  채  총  계', '자본금', '이익잉여금(결손금）', '자본총계', '매출액', '판매비와관리비',
       '영업이익（손실）', '법인세비용차감전순손익', '법인세비용', '당기순이익(손실)', '기업순이익률(%)',
       '유보액/총자산(%)', '유보액/납입자본(%)', '매출액총이익률(%)', '매출액영업이익률(%)', '매출액순이익률(%)',
       '수지비율(%)', '경상수지비율', '영업비율(%)', '금융비용대매출액비율(%', '금융비용대부채비율(%)',
       '금융비용대총비용비율(%', '부채비율(%)', '차입금의존도(%)', '자기자본비율(%)', '순운전자본비율(%)',
       '유동부채비율(%)', '비유동부채비율(%)', '부채총계대 매출액(%)', '총자본회전율(회)', '재고자산회전율(회)',
       '매출채권회전율(회)', '매입채무회전율(회)', '미수금', '매출원가', '무형자산', '재고자산'],
      dtype='object')

In [11]:
isinactive = non_financial[['사업자등록번호', '휴폐업 유무']]
isinactive

,사업자등록번호,휴폐업 유무
0,1138111739,N
1,2078132193,N
2,1318167565,N
3,5048128251,N
4,1098164260,N
...,...,...
234367,4808501629,Y
234368,8018501578,Y
234369,4118523419,Y
234370,1738102533,Y


In [12]:
financial = pd.merge(financial, isinactive, left_on='사업자번호', right_on= '사업자등록번호', how = 'left')


In [16]:
financial.columns

Index(['사업자번호', '결산년월', '유동자산', '매출채권', '비유동자산', '유형자산', '자산총계', '유동부채',
       '비유동부채', '부  채  총  계', '자본금', '이익잉여금(결손금）', '자본총계', '매출액', '판매비와관리비',
       '영업이익（손실）', '법인세비용차감전순손익', '법인세비용', '당기순이익(손실)', '기업순이익률(%)',
       '유보액/총자산(%)', '유보액/납입자본(%)', '매출액총이익률(%)', '매출액영업이익률(%)', '매출액순이익률(%)',
       '수지비율(%)', '경상수지비율', '영업비율(%)', '금융비용대매출액비율(%', '금융비용대부채비율(%)',
       '금융비용대총비용비율(%', '부채비율(%)', '차입금의존도(%)', '자기자본비율(%)', '순운전자본비율(%)',
       '유동부채비율(%)', '비유동부채비율(%)', '부채총계대 매출액(%)', '총자본회전율(회)', '재고자산회전율(회)',
       '매출채권회전율(회)', '매입채무회전율(회)', '미수금', '매출원가', '무형자산', '재고자산', '사업자등록번호',
       '휴폐업 유무'],
      dtype='object')

In [15]:
financial['결산년월'].value_counts()

20181231    53453
20191231    49961
20211231    49061
20201231    48105
20180331      707
20190331      669
20210331      621
20200331      609
20190630      554
20180630      552
20220331      547
20200630      531
20210630      496
20190930      245
20180930      237
20200930      226
20210930      221
20220630      100
20201031       54
20211031       52
20191031       51
20180531       49
20180228       46
20210531       46
20220531       46
20200229       46
20190531       46
20190228       46
20210430       44
20180430       43
20190430       43
20210228       39
20181031       39
20200531       38
20200430       37
20220430       32
20220228       30
20210131       28
20211130       26
20180831       23
20190831       23
20210831       22
20210731       22
20191130       21
20200131       21
20200831       21
20220131       20
20201130       20
20181130       18
20200731       16
20180131       15
20190131       11
20180731        9
20190731        9
Name: 결산년월, dtype: int64